# Creación de la Base de Datos.

Ahora llega de momento de separar las bases de datos por tipo. 
- Absoluto
- Porcentaje



In [7]:
import pandas as pd
import os

# 
tidy_path = "..\\tidy_data\\SIM\\"
# Extraemos el nombre de los archivos.
tidy_files = os.listdir(tidy_path)
td ={file: os.path.getctime(tidy_path +"\\" + file) for file in tidy_files}

sorted_tidy = dict(sorted(td.items(), key=lambda item:item[1]))
sorted_tidy

tidy_files = list(sorted_tidy.keys())

print(tidy_files)

['ComposicionTerritorial.xlsx', 'UbicacionGeografica.xlsx', 'DinamicaPoblacional2020.xlsx', 'EstructuraSexo2020.xlsx', 'GruposVulnerables2020.xlsx', 'HechosVitales2020.xlsx', 'Salud2022.xlsx', 'Educacion2020.xlsx', 'HogaresViviendas2020.xlsx', 'MedioAmbiente2020.xlsx', 'EntornoUrbano2020.xlsx', 'Estructuras2021.xlsx', 'Automoviles2022.xlsx', 'AccidentesTransito2022.xlsx', 'SeguridadSep2023.xlsx', 'Economia2021.xlsx', 'FinanzasMunicipales2021.xlsx']


Ahora veamos un ejemplo. 

In [8]:
ex = pd.read_excel(tidy_path + "\\" + tidy_files[0])
ex.set_index(ex.columns[0], drop=True, inplace=True)
descriptor = pd.read_excel("..\\raw_data\\descriptor.xlsx")

stop = 'Aconchi'

i = 0
while ex.index[i] != stop:
    i = i + 1

ex_i = ex.iloc[i:,:]

ex_i

,Localidades,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,AGEB,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Manzanas,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
Unnamed: 0,,,,,,,,,,,,,,,,,,
Aconchi,25,-,1,4,24,96,14,-,11,78.6,3,21.4,223,-,135,60.5,88,39.5
Agua Prieta,261,-,1,0.4,260,99.6,209,-,179,85.6,30,14.4,4115,-,3998,97.2,117,2.8
Alamos,529,-,1,0.2,528,99.8,67,-,14,20.9,53,79.1,2270,-,425,18.7,1845,81.3
Altar,351,-,1,0.3,350,99.7,68,-,31,45.6,37,54.4,394,-,341,86.5,53,13.5
Arivechi,97,-,1,1,96,99,11,-,5,45.5,6,54.5,182,-,65,35.7,117,64.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villa Pesqueira,233,-,1,0.4,232,99.6,16,-,9,56.3,7,43.8,146,-,86,58.9,60,41.1
Yécora,478,-,1,0.2,477,99.8,30,-,9,30,21,70,431,-,172,39.9,259,60.1
Gral. Plutarco Elías Calles,202,-,1,0.5,201,99.5,82,-,54,65.9,28,34.1,1075,-,802,74.6,273,25.4


Ahora lo aplicamos a todas las bases de datos.

In [9]:

def new_dfs(files: list):
    dfs = []
    for f in files:
        ex = pd.read_excel(tidy_path + "\\" + f)
        ex.set_index(ex.columns[0], drop=True, inplace=True)
        stop = 'Aconchi'
        i = 0
        while ex.index[i] != stop:
            i = i + 1
        ex_i = ex.iloc[i:,:]
        dfs.append(ex_i)
    return dfs

data = new_dfs(tidy_files)
len(data)

17

Ahora, vamos a leer los descriptores ya construidos. 

In [10]:
import numpy as np

sim_cols = list(descriptor['Origen'].unique())
sim_cols.remove(np.nan)
descriptor['Origen']= descriptor['Origen'].ffill()
descriptor = descriptor.set_index(['Origen','Némonico'],drop=True)
sim_cols


['Composición Territorial ',
 'Ubicación Geográfica',
 'Dinámica Poblacional',
 'Estructura Edad y Sexo',
 'Poblacion Vulnerable',
 'Hechos Vitales',
 'Salud',
 'Educación',
 'Hogares y Vivienda',
 'Medio Ambiente',
 'Entorno Urbano',
 'Estructuras',
 'Transporte',
 'Accidentes de Transito',
 'Seguridad',
 'Economia',
 'Finanzas Municipales']

Creamos una herramienta para seleccionar secciones a pesar del indice multiple. 

In [13]:
def select_level_one(level_name, df):
    return df.loc[(level_name, slice(None)), :]

Nivel 0                           Nivel 1 Nivel 2  \
Origen     Némonico                                                         
Transporte TPETOT    Transporte                             Total     NaN   
           TPEATO           NaN                       Automóviles     NaN   
           TPECPJ           NaN           Camiones para pasajeros     NaN   
           TPECCC           NaN  Camiones y camionetas para carga     NaN   
           TPMOTO           NaN                      Motocicletas     NaN   

                    Unidades Fecha de Captura  
Origen     Némonico                            
Transporte TPETOT        NaN             2022  
           TPEATO        NaN             2022  
           TPECPJ        NaN             2022  
           TPECCC        NaN             2022  
           TPMOTO        NaN              NaN

Ahora, si procedemos. 

In [14]:
sim_m = []
data_m = []
ver_f = []
for k, col in enumerate(sim_cols):
    sim_m.append(select_level_one(col, descriptor).shape[0])
    data_m.append(data[k].shape[1])
    if 2 * sim_m[k] != data_m[k]:
        ver_f.append(k)
    else:
        ver_f.append(-1)
print(sim_m)
print(data_m)
print(ver_f)

data[12]
# select_level_one(sim_cols[12],descriptor)

[9, 9, 24, 36, 10, 8, 8, 20, 4, 6, 22, 29, 5, 4, 1, 19, 23]
[18, 10, 48, 36, 20, 16, 16, 40, 10, 12, 44, 29, 10, 8, 2, 38, 46]
[-1, 1, -1, 3, -1, -1, -1, -1, 8, -1, -1, 11, -1, -1, -1, -1, -1]


,Total,Unnamed: 2,Automóviles,Unnamed: 4,Camiones para pasajeros,Unnamed: 6,Camiones y camionetas para carga,Unnamed: 8,Motocicletas,Unnamed: 10
Unnamed: 0,,,,,,,,,,
Aconchi,936,100,437,46.7,4,0.4,480,51.3,15,1.6
Agua Prieta,48087,100,31678,65.9,226,0.5,15774,32.8,409,0.9
Alamos,4627,100,1918,41.5,46,1,2642,57.1,21,0.5
Altar,2799,100,1541,55.1,9,0.3,1240,44.3,9,0.3
Arivechi,321,100,99,30.8,1,0.3,219,68.2,2,0.6
...,...,...,...,...,...,...,...,...,...,...
Villa Pesqueira,435,100,156,35.9,3,0.7,267,61.4,9,2.1
Yécora,1127,100,323,28.7,4,0.4,796,70.6,4,0.4
Gral. Plutarco Elías Calles,1928,100,948,49.2,1,0.1,955,49.5,24,1.2
